In [3]:
using Images,Flux,BSON

In [15]:
# Function that loads images from classified
# folder to the dataset
function createDataset(path)
    X = []
    y = []
    for label in readdir(path)
        for file in readdir("$path/$label")
            img = load("$path/$label/$file")
            data = reshape(Float32.(channelview(img)),28,28,1)
            if length(X) == 0
                X = data
            else
                X = cat(X,data,dims=3)
            end
            push!(y,parse(Float32,label))
        end
    end
    return X,y
end;

In [14]:
# Load training and testing datasets from files
x_train,y_train = createDataset("training");
x_test,y_test = createDataset("testing");

In [11]:
# Reshape the data
x_train = reshape(x_train, 28, 28, 1, :);
x_test = reshape(x_test, 28, 28, 1, :);

In [12]:
# Define the Lenet v5 model
model = Chain(
    Conv((5,5),1 => 6, relu),
    MaxPool((2,2)),
    Conv((5,5),6 => 16, relu),
    MaxPool((2,2)),
    Flux.flatten,
    Dense(256=>120,relu),
    Dense(120=>84, relu),
    Dense(84=>10, sigmoid),
    softmax
);

In [13]:
# Function that measures the accuracy of
# model on testing dataset
function accuracy()
	correct = 0
	for index in 1:length(y_test)
    	probs = model(Flux.unsqueeze(x_test[:,:,:,index],dims=4))
        predicted_digit = argmax(probs)[1]-1
    	if predicted_digit == y_test[index]
        	correct +=1
        end
    end
    return correct/length(y_test)
end;

In [34]:
# Assemble the training data
train_data = Flux.DataLoader((x_train,y_train), shuffle=true, batchsize=64)

# Initialize the ADAM optimizer with default settings
optimizer = Flux.setup(Adam(),model)

# Define the loss function that uses the cross-entropy to 
# measure the error by comparing model predictions of data 
# row "x" with true data from labels vector "y"
function loss(model, x, y)
	return Flux.crossentropy(model(x),Flux.onehotbatch(y,0:9))
end

# Train the model 10 times in a loop
# and display the accuracy on each
# iteration
for epoch in 1:10
	Flux.train!(loss, model, train_data, optimizer)
    println(accuracy())
end

0.982
0.9868
0.9855
0.9878
0.9867
0.9881
0.9891
0.9889
0.9895
0.9846


In [10]:
# Save trained model to a file
BSON.@save "digits.bson" model